In [1]:
import dlc_bci as bci

In [2]:
train_input , train_target = bci.load(root = './data_bci', one_khz=True)

print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])


In [3]:
test_input, test_target = bci.load(root = './data_bci', train=False, one_khz=True)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [4]:
train_input[0].shape

torch.Size([28, 500])

In [5]:
import torch
from torch.autograd import Variable
from torch.nn import functional as F

<h3>Stardardize the data</h3>

<h3>1. normalize data channel-wise</h3>

In [6]:
right_indices = train_target.nonzero()

In [7]:
left_indices = (train_target == 0).nonzero()

In [8]:
# reshape from [157, 1] to [157]
right_indices = right_indices.view(-1)
left_indices = left_indices.view(-1)

In [9]:
right_data = torch.index_select(train_input, 0, right_indices)
left_data = torch.index_select(train_input, 0, left_indices)

print(right_data.shape, left_data.shape)

torch.Size([157, 28, 500]) torch.Size([159, 28, 500])


In [10]:
right_data = torch.nn.functional.normalize(right_data, p=2, dim=0)
left_data = torch.nn.functional.normalize(left_data, p=2, dim=0)

In [11]:
train_input.shape

torch.Size([316, 28, 500])

In [6]:
n_channels = 32
kernel_size = 3
n_hidden = 128
input_shape = train_input.shape
ff1_in = n_channels * (input_shape[2] - 2) // 2


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv1d(input_shape[1], n_channels, kernel_size)
        self.max_pool = torch.nn.MaxPool1d(2)
        self.fc1 = torch.nn.Linear(ff1_in, n_hidden)
        self.fc2 = torch.nn.Linear(n_hidden, 2)
        self.sm = torch.nn.Softmax(1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        x = x.view(x.shape[0], ff1_in)
        x = F.relu(self.fc1(x))
        x = self.sm(self.fc2(x))
        return x

In [ ]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new(target.size(0), target.max() + 1).fill_(0)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [ ]:
#train_target = convert_to_one_hot_labels(train_input, train_target)

In [ ]:
# merge back right and left data into one data-set
train_input_norm = torch.FloatTensor(train_input.shape).fill_(0)
print(train_input_norm.shape)

In [ ]:
right_data.shape

In [ ]:
for i, ind in enumerate(right_indices):
    train_input_norm[ind] = right_data[i]
    
for i, ind in enumerate(left_indices):
    train_input_norm[ind] = left_data[i]

In [7]:
def train_model(model, train_input, train_target, mini_batch_size):
    criterion = torch.nn.CrossEntropyLoss()
    eta = 1e-1

    for e in range(0, 25):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.data[0]
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print(e, sum_loss)

In [8]:
def compute_nb_errors(model, input, target, mini_batch_size):

    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        predicted_classes = output.data.max(1)[1]
        true_classes = target.data.narrow(0, b, mini_batch_size)
        nb_errors += (predicted_classes == true_classes).sum()

    return nb_errors

In [9]:
mini_batch_size = 4

In [ ]:
test_input_norm = torch.nn.functional.normalize(test_input, p=2, dim=0)
#test_target = convert_to_one_hot_labels(test_input, test_target)

In [10]:
train_target = train_target.long()
test_target = test_target.long()

In [11]:
train_input_norm, train_target = Variable(train_input_norm), Variable(train_target)
test_input_norm, test_target = Variable(test_input_norm), Variable(test_target)

In [12]:
for k in range(0, 10):
    model = Net()
    train_model(model, train_input_norm, train_target, mini_batch_size) 
    nb_test_errors = compute_nb_errors(model, test_input_norm, test_target, mini_batch_size)
    print('test error Net {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                       nb_test_errors, test_input.size(0)))

0 64.39027827978134
1 64.49767112731934
2 64.49767112731934
3 64.49767112731934
4 64.49767112731934
5 64.49767112731934
6 64.49767112731934
7 64.49767112731934
8 64.49767112731934
9 64.49767112731934
10 64.49767112731934
11 64.49767112731934
12 64.49767112731934
13 64.49767112731934
14 64.49767112731934
15 64.49767112731934
16 64.49767112731934
17 64.49767112731934
18 64.49767112731934
19 64.49767112731934
20 64.49767112731934
21 64.49767112731934
22 64.49767112731934
23 64.49767112731934
24 64.49767112731934
test error Net 51.00%% 51/100
0 64.87638062238693
1 64.7476709485054
2 64.7476709485054
3 64.7476709485054
4 64.7476709485054
5 64.7476709485054
6 64.7476709485054
7 64.7476709485054
8 64.7476709485054
9 64.7476709485054
10 64.7476709485054
11 64.7476709485054
12 64.7476709485054
13 64.7476709485054
14 64.7476709485054
15 64.7476709485054
16 64.7476709485054
17 64.7476709485054
18 64.7476709485054
19 64.7476709485054
20 64.7476709485054
21 64.7476709485054
22 64.7476709485054
23 6

KeyboardInterrupt: 